In [23]:
# Importing the RISM instrumentation json file into ElasticSearch
# This script requires that you ran preprocess_rism.ipynb first
# To use yourself:
# 1. Update the RISM_DIR variable to point to the directory of your RISM json file
# 2. If necessary, update the RISM_JSON variable with its filename

import os
import re
import json
from elasticsearch import Elasticsearch,helpers
import elasticsearch_dsl

CUR_DIR = os.getcwd()
RISM_DIR = os.path.join(CUR_DIR, '../hamr_2018/rismAllMARCXML')
RISM_JSON = 'rism_instronly.json'

In [24]:
# open the json file and parse its contents

with open(os.path.join(RISM_DIR, RISM_JSON), 'r') as f:
    all_records = json.load(f)

print('I have a list of ', len(all_records), 'JSON records')
    
    

I have a list of  1003148 JSON records


In [31]:
# Now, create an ELK stack on your machine. Refer to https://elk-docker.readthedocs.io/ for more info.
# Make sure to increase the RAM available to Docker if necessary - 4GB is needed at least

# To install for the first time, open a terminal and run: 
# `sudo docker run -p 5601:5601 -p 9200:9200 -p 5044:5044 -it --name elk sebp/elk`

# If already installed, open a terminal and run:
# `sudo docker start elk`

# In the browser, go to: http://localhost:5601 for Kibana
# to login to the Docker itself, open a terminal and run: 
# `sudo docker exec -it elk /bin/bash`

ELASTIC_HOST = 'localhost:9200'
ELASTIC_INDEX = 'rism'   # create a new index with any name, you choose


In [ ]:
# Import the JSON in elastic
# This might easily take an hour I guess, for the full 1 million records in RISM

es = Elasticsearch([ ELASTIC_HOST ])
fewrecords = all_records[0:10]
print(fewrecords)


# first create an index to upload this doc to (? necessary ?)
#if not es.indices.exists(index= ELASTIC_INDEX )
es.indices.create(ELASTIC_INDEX, ignore=400)
    
# then bulk index
# helpers.bulk(es,fewrecords)

myid=1
for myrecord in all_records:
    # print(myrecord)
    es.index(index=ELASTIC_INDEX, doc_type='doc', id=myid, body=myrecord)
    myid += 1
    if not myid%10000:
        print ('Inserted ', myid-1, ' documents')
    
print ('Finished: in total inserted ', myid-1, ' documents')    

[{'id': '000051650', 'instr_summ': ['S', 'strings', 'cor (2)'], 'instr_full': ['S', 'vl (2)', 'b', 'cor (2)']}, {'id': '000051651', 'instr_summ': ['S', 'orch'], 'instr_full': ['S', 'vl (2)', 'vla', 'b', 'ob', 'fag', 'cor']}, {'id': '000051652', 'instr_summ': ['S', 'orch'], 'instr_full': ['S', 'vl (2)', 'vla (1)', 'vla', 'b', 'ob (2)', 'cl (2)', 'cor (2)']}, {'id': '000051653', 'instr_summ': ['V (2)', 'orch'], 'instr_full': ['S', 'B', 'vl (2)', 'vla', 'b', 'fag', 'cor (2)', 'tr (2)']}, {'id': '000051654', 'instr_summ': ['S', 'strings'], 'instr_full': ['S', 'vl (2)', 'vla', 'b']}, {'id': '000051655', 'instr_summ': ['S', 'orch'], 'instr_full': ['S', 'vl (2)', 'vla', 'b', 'ob', 'cor']}, {'id': '000051656', 'instr_summ': ['S', 'orch'], 'instr_full': ['S', 'vl (2)', 'vla', 'b', 'ob', 'cor']}, {'id': '000051657', 'instr_summ': ['S', 'orch'], 'instr_full': ['S', 'vl (2)', 'violetta', 'b', 'ob (2)', 'fag', 'cor']}, {'id': '000051658', 'instr_summ': ['V (2)', 'orch'], 'instr_full': ['S', 'B', 'v

In [ ]:
# In Kibana, try to create a Dashboard to visualize the data.

# Follow this tutorial: https://www.elastic.co/guide/en/kibana/6.4/tutorial-build-dashboard.html 

# (Go to Management, create a new index pattern to connect to the newly created index)
